# SETUP for
# Autonomous Class Attendance System using Face Recognition

Importing Modules
--------------

In [1]:
#FOr dataset creator
import numpy as np
import cv2
from matplotlib import pyplot as plt

#For Trainer
from PIL import Image
import os

#For data updations
import pandas as pd

Managing data
------------------------------------------

Reseting the database

In [2]:
#used to reset the data and image dataset
def reset(x):
    #cross checking to format all information, just for precaution
    if x=="admin":
        #initialization value
        x=[]
        y=[[0,"Dummy"]]
        
        #importing the data frame from csv file
        dataframex=pd.DataFrame(x,columns=["Date"])
        dataframey=pd.DataFrame(y,columns=['Roll','Name'])
        
        #writting back blank value in the csv file
        dataframex.to_csv ("attendance.csv",index=False)
        dataframey.to_csv ("roll.csv",index=False)
        
        #success feedback for data (csv)
        print("Data reset complete")
        
        #assingning dataset path 
        d='dataset'
        
        #selecting taget
        filesToRemove = [os.path.join(d,f) for f in os.listdir(d)]
        
        #iterating removal process for all file present
        for f in filesToRemove:
            
            #delection process
            os.remove(f)
        
        #success feedback for dataset (image)
        print("Database reset complete")
    
    #incase of wrong keyword
    else:
        
        #failure feedback
        print("Failed to reset")

In [3]:
#used to set new attributes in the csv data
def setRec(id,name):
    
    #importing the data frame from csv file
    x=pd.read_csv("attendance.csv")
    y=pd.read_csv("roll.csv")
    
    #adding a student and roll no sheet
    a=[id,name]
    #
    m=y.append({"Roll":id,"Name":name},ignore_index=True)
    #writting back to csv file
    m.to_csv("roll.csv",index=False)
    
    #adding a student in attendance sheet
    x.update({id:[]})
    #writting back to csv file
    x.to_csv("attendance.csv",index=False)

Checking the camera
------------------------------

In [4]:
#checking the camera
def camera():
    
    #imprting the openCV cascade for frontal face detection
    faceDetect = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
    
    #setting the input source to the webcam
    cam=cv2.VideoCapture(0)
    
    #running the image capturing window
    while(True):
        
        #reading the frame
        rett,img=cam.read()
        
        #convering the color image into grayscale as the cascade only understand grayscale value
        gray=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
        
        #getting the multiscale value
        faces=faceDetect.detectMultiScale(gray,1.3,5)
        
        #putting a rectangle frame on face
        for(x,y,w,h) in faces:
            
            #putting the rectangle frame on every face
            cv2.rectangle(img,(x,y),(x+w,y+h),(0,0,255),2)
        
        #starting webcam
        cv2.imshow("Face",img)
        
        #stopping the camera
        if (cv2.waitKey(1)==ord("q")):
            break
            
    #closing the camera window
    cam.release()
    cv2.destroyWindow("Face")


Training the machine with data
------------

In [5]:
#associating image with id
def getImagesWithId(path):
    
    #setting the image path
    imagePaths=[os.path.join(path,f) for f in os.listdir(path)]
    
    #initializing the values
    faces=[]
    IDs=[]
    
    #iterating for every image in the directory
    for imagePath in imagePaths:
        
        #getting the picture data from path and converting it into 8 bit black and white (L)
        faceImg=Image.open(imagePath).convert("L")
        
        #converting the pic into a numpy array in integer data type (8 bit)
        faceNp=np.array(faceImg,"uint8")
        
        #getting back the student id from the photo name
        ID=int(os.path.split(imagePath)[-1].split(".")[1])
        
        #appending the face matrix in the list
        faces.append(faceNp)
        
        #appending the id values in the list
        IDs.append(ID)
        
        #opening the training window
        cv2.imshow("training",faceNp)
        cv2.waitKey(10)
    
    #returning the id and face list
    return np.array(IDs), faces

In [6]:
#used to train the model with images
def trainMachine():
    
    ##getting the cascade information from openCV
    recognizer=cv2.face.LBPHFaceRecognizer_create()
    
    #setting the dataset path
    path="dataset/"

    #geting the id and associated images
    Ids,faces=getImagesWithId(path)
    
    #traing with image data and ralting with id
    recognizer.train(faces,Ids)
    
    #saving the training infromation in location
    recognizer.save("recognizer/trainingData.yml")
    
    #closing the training window
    cv2.destroyWindow("training")

In [7]:
#used to collect the sample images i.e. used to train the machine
def train():
    #getting the cascade information from openCV
    faceDetect = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
    
    #getting the image source from web camera
    cam=cv2.VideoCapture(0)
    
    #initializing sample no, It is used to name the sample photos
    sampleNum=0
    
    #getting the dataframe from csv
    y=pd.read_csv("roll.csv")
    
    #setting the id value as the no of columns in the dataframe
    #to assign new student we have to increase the value of id
    id=y.shape[0]
    
    #print the id value of the new student
    print(id)
    
    #getting the student name
    name=input("Enter student name : ")
    
    #updating the student id and name in the csv
    setRec(id,name)

    #running the image capturing window
    while(True):
        
        #reading the frame
        rett,img=cam.read()
        
        #convering the color image into grayscale as the cascade only understand grayscale value
        gray=cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        
        #getting the multiscale value
        faces=faceDetect.detectMultiScale(gray,1.3,5)
        
        #collecting the sample images and putting a rectangle frame on face 
        for(x,y,w,h) in faces:
            
            #increasing the sample number with each iteration
            sampleNum=sampleNum+1
            
            #storing the sample images
            cv2.imwrite("dataset/User."+str(id)+"."+str(sampleNum)+".jpg",gray[y:y+h,x:x+w])
            
            #showing the rectangle
            cv2.rectangle(img,(x,y),(x+w,y+h),(0,0,255),2)
            cv2.waitKey(100)
        
        #starting the webcam window
        cv2.imshow("face",img)
        
        #putting regular interval between two samples to take diverse images
        cv2.waitKey(2)
        
        #stopping the capturing process after collection of 30 successful face samples
        if (sampleNum>30):
            break
    
    
    
    
    #closing camera window
    cam.release()
    cv2.destroyWindow("face")
    cv2.waitKey(1)
    
    #training the machine
    trainMachine()


Recognizer
-----------

In [8]:
#this will recognize the face with name of the preson
def recognize():
    
    #getting the cascade information from openCV
    faceDetect = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
    
    #getting the image source from web camera
    cam=cv2.VideoCapture(0)
    rec=cv2.face.LBPHFaceRecognizer_create()
    
    #declaring font of the text in the frame
    font=cv2.FONT_HERSHEY_COMPLEX

    #reading the training data
    rec.read("recognizer/trainingData.yml")
    
    #initializing the id
    id=0
    
    #get the student list
    y=pd.read_csv("roll.csv")
    stuList=y["Name"].values.tolist()

    #running the image capturing window
    while(True):
        
        #reading the frame
        rett,img=cam.read()
        
        #convering the color image into grayscale as the cascade only understand grayscale value
        gray=cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        
        #getting the multiscale value
        faces=faceDetect.detectMultiScale(gray,1.3,5)
        
        #putting a rectangle frame on face
        for(x,y,w,h) in faces:
            
            #plotting the rectangle
            cv2.rectangle(img,(x,y),(x+w,y+h),(0,0,255),2)
            #predicting student id from the face in rectangle
            id,conf=rec.predict(gray[y:y+h,x:x+w])
            
            #getting student name from student id
            id=stuList[id]
            
            #putting the name in the rectangle
            cv2.putText(img,str(id),(x,y+h),font,0.55,(0,255,0),1)
        
        #starting webcam
        cv2.imshow("Face",img)
        
        #stopping camera
        if (cv2.waitKey(1)==ord("q")):
            break
    
    #closing camera window
    cam.release()
    cv2.destroyWindow("Face")

# Instruction


1. To check camera and face detection, call : camera()
2. To train the machine with a new person, call : train()
3. To check face detection and recognition, call : recognize()


>> To reset the data, call: reset(**password**)

**NB: to close/stop capturing process press "q"**

# Test

In [9]:
#test the camera
#press "q" to quit

camera()

In [13]:
#test the face recognisation with name
#press "q" to quit

recognize()

# Modifying Data


In [12]:
#train the machine with new persoin with name
#test the camera first before training the data
#train the machine with sufficient lightening condition
#avoid quick moment
#avoid the presence of any person or face in the background, the machine may predict incorrectly
#the auto generated student no will be printed 

#press "q" to quit

#remove the "#" to run

##train()

1
Enter student name : sid


In [11]:
#Fully reset the student data, image data and attendance sheet
#Avoid running the code
#password is "admin"

#remove the "#" to run

##reset("admin")

Data reset complete
Database reset complete
